# Remote execution on compute cluster

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core import ScriptRunConfig

ws = Workspace.from_config()
target = ws.compute_targets['cpu-cluster']

script = ScriptRunConfig(
    source_directory='030_scripts',
    script='sklearn_vanilla_train.py',
    compute_target=target,
    environment = ws.environments['AzureML-sklearn-0.24-ubuntu18.04-py37-cpu'],
    arguments=['--alpha', 0.01]
)

exp = Experiment(ws, 'remote-script-execution')
run = exp.submit(script)
run.wait_for_completion(show_output=True)

## Custom environment

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
import sklearn

diabetes_env = Environment(name="diabetes-training-env")
diabetes_env.python.conda_dependencies = CondaDependencies.create(
                                            conda_packages=[
                                                f"scikit-learn=={sklearn.__version__}",
                                                "mlflow",
                                            ],
                                            pip_packages=[
                                                "azureml-defaults",
                                                "azureml-mlflow",
                                                "azureml-dataprep[pandas]"
                                            ])
# Or if you had a yml conda file
# diabetes_env = Environment.from_conda_specification(
#                            name = "diabetes-training-env",
#                            file_path = "diabetes-conda.yml")
# Or even from Docker file
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py#from-dockerfile-name--dockerfile--conda-specification-none--pip-requirements-none-

In [ ]:
diabetes_env.environment_variables["MY_VAR"] = "Hello from environment"

In [ ]:
script = ScriptRunConfig(
    source_directory='030_scripts',
    script='sklearn_vanilla_train.py',
    compute_target=target,
    environment = diabetes_env,
    arguments=['--alpha', 0.01]
)

exp = Experiment(ws, 'remote-script-execution')
run = exp.submit(script)
# First time you will see 20_image_build_log.txt.
# The image will be stored in the container registry and will
# be reused in follow up calls.
run.wait_for_completion(show_output=True)

In [ ]:
# Optionally, register the environment
diabetes_env.register(ws)